<a href="https://colab.research.google.com/github/carlosks/carlosks/blob/main/finetuning_summarizer_Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-yc1ypmy9/unsloth_4de3e815efbc4c1db93bdd1214d061e4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-yc1ypmy9/unsloth_4de3e815efbc4c1db93bdd1214d061e4
  Resolved https://github.com/unslothai/unsloth.git to commit 0fbbdfc091fc1a3b1c09b752794963681d10fad2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.8.6-py3-none-any.whl (245 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.11.0
    Uninstalling trl-0.11.0:
      Successfully uninstalled trl-0.11.0


In [3]:
!pip install --upgrade accelerate
!pip install --upgrade transformers
!pip install --upgrade peft
!pip install --upgrade bitsandbytes
!pip install --upgrade trl
!pip install --upgrade datasets


  Using cached trl-0.11.0-py3-none-any.whl.metadata (12 kB)
Using cached trl-0.11.0-py3-none-any.whl (316 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.8.6
    Uninstalling trl-0.8.6:
      Successfully uninstalled trl-0.8.6


In [ ]:
!pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.2 MB/s eta 0:00:00


In [4]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/drive/MyDrive/Arquivos_Aula_1_Fine_Tunning/news_dataset_chat_data.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/formatted_news_dataset_chat_data.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [5]:
def format_dataset_into_model_input(data):
    def separate_text(full_text):
        news_start = full_text.find("[|News|]") + len("[|News|]")
        news_end = full_text.find("[|eNews|]")
        summary_start = full_text.find("[|summary|]") + len("[|summary|]")
        summary_end = full_text.find("[|esummary|]")

        instruction = full_text.split('\n')[0]
        input_text = full_text[news_start:news_end].strip()
        response = full_text[summary_start:summary_end].strip()

        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    # Iterate over the list of dictionaries
    try: # this will allow the program to continue if there is a KeyboardInterrupt
        for item in data:
            # Access the 'input' key for each dictionary
            for prompt in item['input']:
                instruction, input_text, response = separate_text(prompt)
                instructions.append(instruction)
                inputs.append(input_text)
                outputs.append(response)
    except KeyboardInterrupt:
        pass # this will ignore the KeyboardInterrupt and allow the program to continue

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON

In [6]:
# Load the dataset
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

format_dataset_into_model_input(data)


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/formatted_news_dataset_chat_data.json"

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.814900
2,0.814900
3,0.777400
4,0.674700
5,0.547400
6,0.399100
7,0.271500
8,0.153600
9,0.067400
10,0.054800


In [ ]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS NEWS",
        "Following the death of the Danish butter magnate Lars Emil Bruun in 1923, his will had a curious order: His vast accumulation of coins, notes and medals, amassed over more than six decades, should be held as an emergency reserve for Denmark’s national collection in case it were ever destroyed. After a century, if all was well, his own cache could finally be sold to benefit his descendants. On Tuesday, just under a year since the 100-year-old order expired, the first set of coins from Bruun’s personal 20,000-piece collection went up for auction in Copenhagen. After nearly eight hours of bidding, the opening 286 lots sold for a combined 14.82 million euros ($16.5 million). It will take several more sales to empty Bruun’s coffers, but once completed, it will be the most expensive international coin collection ever sold, according to Stack’s Bowers, the rare coin dealer and auction house hosting the sales. The L.E. Bruun Collection was insured for 500 million Danish kroner, or around $72.5 million. The auction house described it as the most valuable collection of world coins to ever come to market. In a press release following Tuesday’s auction, Stack’s Bowers Galleries’ president, Brian Kendrella, described the first sale as “truly a landmark event for the world coin market.” Where the numismatist’s collection has resided over the past century had been something of a mystery, its location known to few.But Bruun believed that hiding his treasure was for a noble cause; following the destructionhe saw of World War I, he feared the Royal Danish Coin and Medal Collection could one day face bombing or looting, according to the auction house. Bruun began collecting currency as a child in 1859, when his uncle died and named him among the recipients of some of his coins, according to the sales catalog. The son of innkeepers and landowners, he learned in his 20sthat his family inheritance had been squandered and he was saddled in debt. He began his own business in butter with a loan, eventually earning a fortune from sales and exports. With his wealth, he became a prolific coin collector, and was a founding member of the Danish Numismatic Society in 1885. “The good thing about collecting coins is that when you are upset about something or you feel unsettled, then you go and look at your coins, and then you calm down by studying them again and again pondering the many unsolved problems they present,” he once told a Danish magazine, per the catalog. “People who are exclusively devoted to their business make a great mistake. I, for one, could never imagine thinking about nothing but butter until my dying days.” Tuesday’s sale included gold and silver coins from Denmark, Norway and Sweden, dated from the late 15th century to the latter years of Bruun’s life. The star lot was one of Scandinavia’s oldest gold coins, according to the catalog a noble of King Hans dated from 1496. The coin smashed auction house estimates to fetch 1.2 million euros ($1.34 million), setting a new world record for a Scandinavian coin at auction, Stack’s Bowers Galleries said. “Hands-down my favorite piece in the sale is the 1496 gold noble of King Hans, who was king of Denmark and Norway under the Kalmar Union, as well as Sweden for a brief time,” said Matt Orsini, director of world and ancient numismatics at Stack’s Bowers Galleries, in a press statement prior to the first auction. “It is important on so many levels — it’s the very first gold coin struck by Denmark, it’s the very first dated coin struck by the Danish kingdom, and it’s unique in private hands.” Over the past few months, the coins toured different fairs and exhibited at Stack’s Bowers’ galleries. They were also put on display in Copenhagen just before the sale.[|eNews|]\n\n[|summary|]After a century, the first set of coins from Danish butter magnate Lars Emil Bruun's 20,000-piece collection went up for auction in Copenhagen, selling for a combined 14.82 million euros. The collection, insured for $72.5 million, is expected to be.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nSUMMARIZE THIS NEWS\n\n### Input:\nFollowing the death of the Danish butter magnate Lars Emil Bruun in 1923, his will had a curious order: His vast accumulation of coins, notes and medals, amassed over more than six decades, should be held as an emergency reserve for Denmark’s national collection in case it were ever destroyed. After a century, if all was well, his own cache could finally be sold to benefit his descendants. On Tuesday, just under a year since the 100-year-old order expired, the first set of coins from Bruun’s personal 20,000-piece collection went up for auction in Copenhagen. After nearly eight hours of bidding, the opening 286 lots sold for a combined 14.82 million euros ($16.5 million). It will take several more sales to empty Bruun’s coffers, but once completed, it will b

In [ ]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS NEWS",
        "The aroma of freshly baking flatbread wafts through the air as a unit of British soldiers position themselves for a quick patrol around the village of Sindh Kalay. A British soldier on patrol in the mock Afghan village of Sindh Kalay. Market vendors hawk grapes and melons, as a group of village elders sit smoking water pipes and suspicious-looking men lurk beside battered motorcycles. What should the soldiers do? Conduct a weapons search? Approach the village elders first? In the complex political and cultural terrain of Afghanistan, what is the best course of action? Except this is not Afghanistan. It's Norfolk, England. Instead of the Hindu Kush mountains, it is the green ladscape and tidy farmhouses of the English countryside that stretch out behind them. Welcome to the British Army's state-of-the art training ground. It cost more than $20 million to build and every British soldier serving in Afghanistan will do his or her training here. \"I think it's the closest thing you are going to get short of being in Afghanistan itself,\" says Col. David Colthup of the 2nd Battalion of the Yorkshire Regiment. His troops have already served one tour of duty in Afghanistan's Helmand province and are training for another. British troops serving in Helmand province are tasked with mentoring and training Afghan security forces. Not an easy job in a Taliban stronghold and Afghanistan's center of opium production. \"Ultimately, a soldier joins the army and trains to fight. That's what a soldier trains to do. But today, it's a much, much more complex environment,\" explains Colthup. \"The business of being able to interact either through an interpreter or through Afghan security forces, whether they are police or army. And to understand how the people operate and how we can interact better with them. Because ultimately, that's what it's about,\" he says. The most distinctive features of Sindh Kalay are the high three-meter walls that make up the village compound, creating narrow alleyways difficult for troops to patrol. The village is staffed with Afghan asylum-seekers, many of whom have fled the Taliban. They play the roles of market vendors, village elders and sometimes Afghan security forces. Several Afghan women are also on hand, useful for training British soldiers on the religious and cultural sensitivities of entering an Afghan home.  Watch British troops training in mock Afghan village » The Taliban insurgents are played by Nepalese Ghurkha soldiers authorized to handle weapons. They play their roles silently, unable to partake in the Pashtun banter among the Afghans. Fazel Beria is also an asylum-seeker from Afghanistan. He is responsible for recruiting and for creating the sights and smells of Sindh Kalay and is easily identifiable as the only Afghan in the market in Western clothes. He beams with pride walking down the bazaar and clearly relishes his role in training the British Army. \"Everything with the culture comes up with the issue of hearts and minds,\" he explains. \"If you want to win that, you need to know about their culture. You need to respect their culture, their religion and their way of life.\" He gives high marks to the soldiers training so far. After each exercise, the Afghan actors talk directly to the soldiers about what went wrong and what went right. Sometimes, it's the little things that count. \"Yes, there have been quite a lot of surprises,\" Beria says. Like Afghan will sit cross legged for hours. \"The British soldier cannot do that,\" he laughs. \"The Afghan will be sitting very comfortable and the British soldier is not. So, they have to get used to it.\"  See photos of British troops on patrol in Sindh Kalay -- and for real in Afghanistan » Previously, the army trained on farmhouses and in urban neighborhoods that resembled Northern Ireland more than Afghanistan. But Sindh Kalay does more than mimic the physical reality of Afghanistan. It also mirrors the changing tactics on the ground. Troops are grilled in[|eNews|]\n\n[|summary|]British soldiers train in mock Afghan village before deployment to Afghanistan .\nVillage features Afghan asylum-seekers as vendors, elders, Afghan forces .\nTaliban militants are played by Nepalese Ghurkha soldiers .\nVillage trains soldiers to understand Afghan customs, respect Afghan culture .", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS NEWS

### Input:
The aroma of freshly baking flatbread wafts through the air as a unit of British soldiers position themselves for a quick patrol around the village of Sindh Kalay. A British soldier on patrol in the mock Afghan village of Sindh Kalay. Market vendors hawk grapes and melons, as a group of village elders sit smoking water pipes and suspicious-looking men lurk beside battered motorcycles. What should the soldiers do? Conduct a weapons search? Approach the village elders first? In the complex political and cultural terrain of Afghanistan, what is the best course of action? Except this is not Afghanistan. It's Norfolk, England. Instead of the Hindu Kush mountains, it is the green ladscape and tidy farmhouses of the English countryside that stretch out behind them. Welcom

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/lora_model")


('/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/lora_model/tokenizer.json')

In [ ]:
import json
news_test = []

with open('/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/test_news/news_contents.json', 'r') as news_file:
  news_test = json.loads(news_file.read())['news_content']

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)



==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[
alpaca_prompt.format(
        "SUMMARIZE THIS NEWS",
        news_test[1],
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS NEWS

### Input:
Award-winning Nigerian singer Adekunle Gold rose to international fame when his debut album “Gold” reached number seven on the Billboard World Charts. Since then, the 37-year-old has released four more albums, amassing hundreds of millions of streams worldwide. Behind the scenes, however, he has been quietly battlingsickle cell disease, an inherited blood disorder that leads to abnormally shaped red blood cells, causing severe pain, anemia, and potentially life-threatening complications. It affects children who inherit two copies of the sickle cell gene, one from each parent. The disease is most prevalent in Africa, which accounts for66% of cases worldwide,according to the World Health Organization. Nigeria bears the highest burden, whereas many as 150,000 babiesar